Whisper 모델은 주로 자동 음성 인식(automatic speech recognition, ASR)을 목적으로 합니다. 이 모델은 다양한 언어의 오디오를 텍스트로 변환하는 데 특화됨

- 화자 분류를 수행하고자 한다면, pyAudioAnalysis, pyannote.audio, speaker diarization toolkit (S4D), DiarTK 등과 같은 특화된 라이브러리를 찾아보거나, 화자 분류 기능을 제공하는 다른 Hugging Face 모델 사용 필요


1. 코드 실행 전 런타임 유형 GPU로 변경 필요

In [ ]:
!pip install datasets==1.18.3
!pip install librosa

In [ ]:
!python --version

Python 3.10.12


In [ ]:
#warning 뜨면 한번 더 실행
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


Whisper 모델에 AI hub한국어 음성 데이터 파인튜닝 진행한 모델

https://huggingface.co/SungBeom/whisper-base-ko

> 데이터셋 이름	데이터 샘플 수(train/test)

- 고객응대음성	2067668/21092
- 한국어 음성	620000/3000
- 한국인 대화 음성	2483570/142399
- 자유대화음성(일반남녀)	1886882/263371
- 복지 분야 콜센터 상담데이터	1096704/206470
- 차량내 대화 데이터	2624132/332787
- 명령어 음성(노인남여)	137467/237469

> 전체	10916423(13946시간)/1206588(1474시간)






- transformers 라이브러리는 Hugging Face 팀이 개발한 것으로, 다양한 사전 훈련된 모델을 쉽게 로드하고 사용할 수 있게 해주는 도구입니다. 이 라이브러리는 BERT, GPT-2, T5, Whisper 등과 같은 여러 자연어 처리(NLP) 및 음성 처리 모델들을 포함하고 있습니다.
> Whisper 모델은 OpenAI가 개발한 자동 음성 인식(automatic speech recognition, ASR) 모델로, 원래는 OpenAI의 고유한 도구 및 인터페이스를 통해 사용됩니다. 그러나, Hugging Face는 사용자가 훨씬 더 쉽게 다양한 모델에 접근하고 활용할 수 있도록, Whisper 모델을 포함한 많은 모델들을 자신들의 transformers 라이브러리를 통해 제공하고 있습니다



- AutoProcessor:

AutoProcessor 클래스는 오디오 데이터를 모델이 이해할 수 있는 형식으로 변환하는 데 사용됩니다. 이는 토크나이징(tokenizing), 특징 추출(feature extraction), 오디오 신호의 샘플링 레이트 조정 등의 전처리 과정을 담당합니다.
예를 들어, 원시 오디오 신호를 모델이 예측을 수행할 수 있는 형식의 텐서(tensor)로 변환하는 작업을 포함할 수 있습니다.
- AutoModelForSpeechSeq2Seq:

AutoModelForSpeechSeq2Seq 클래스는 사전 훈련된 음성 인식 모델을 나타냅니다. 이 모델은 변환된 오디오 데이터를 받아서 텍스트로 변환하는 역할을 수행합니다.
이 클래스의 인스턴스는 오디오 입력을 텍스트 출력으로 매핑하는 시퀀스-투-시퀀스(sequence-to-sequence) 작업을 수행합니다.

In [ ]:
pip install pydub

In [ ]:
from pydub import AudioSegment

# 오디오 파일 로드
audio = AudioSegment.from_file("/content/drive/MyDrive/salmonn/audio/test.mp3", format="mp3")

# 다른 형식으로 변환
audio.export("/content/drive/MyDrive/salmonn/audio/output.wav", format="wav")
# audio.export("output.ogg", format="ogg")
# audio.export("/content/output.flac", format="flac")

<_io.BufferedRandom name='/content/drive/MyDrive/salmonn/audio/output.wav'>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

In [ ]:
# 1. 한국어 학습 whisper '성범' Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("automatic-speech-recognition", model="SungBeom/whisper-base-ko")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from transformers import pipeline
import soundfile as sf  # To read audio files

# Path to your audio file (can be .mp3 or .wav)
audio_file_path = "/content/drive/MyDrive/salmonn/audio/output.wav"

# Load the audio file
audio_input, sample_rate = sf.read(audio_file_path)

# Prepare the sample in the format expected by the model
sample = {"raw": audio_input, "sampling_rate": sample_rate}

# Perform the prediction
prediction = pipe(sample, batch_size=8)["text"]

# Output the transcription
print(prediction)

# If you want to return timestamps for the predictions
# Make sure to use a model and pipeline that support timestamp generation
prediction_with_timestamps = pipe(sample, batch_size=8, return_timestamps=True)["chunks"]

# Output the transcription with timestamps
print(prediction_with_timestamps)


혹시 가지던 거 u?


ValueError: ignored

In [ ]:
result = WhisperProcessor(r'/content/drive/MyDrive/salmonn/audio/ycyc.wav')
result

AttributeError: ignored

In [ ]:
dir(pipe)

['__abstractmethods__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_batch_size',
 '_ensure_tensor_on_device',
 '_forward',
 '_forward_params',
 '_num_workers',
 '_postprocess_params',
 '_preprocess_params',
 '_sanitize_parameters',
 'binary_output',
 'call_count',
 'check_model_type',
 'default_input_names',
 'device',
 'device_placement',
 'ensure_tensor_on_device',
 'feature_extractor',
 'forward',
 'framework',
 'get_inference_context',
 'get_iterator',
 'iterate',
 'model',
 'modelcard',
 'postprocess',
 'predict',
 'preprocess',
 'run_multi',
 'run_single',
 'save_pretrained',
 'task',
 'tokenizer',
 'torch_dtype',
 '

In [ ]:
result = pipe('/content/drive/MyDrive/salmonn/audio/test.mp3')
print(result)

{'text': '혹시 가지던 거 u?'}


In [ ]:
from datasets import load_dataset
from datasets import Audio


In [ ]:
example = '/content/drive/MyDrive/salmonn/audio/fltest.flac'
pipe(example["audio"]["array"])

TypeError: ignored

In [ ]:
help(AutoModelForSpeechSeq2Seq)

Help on class AutoModelForSpeechSeq2Seq in module transformers.models.auto.modeling_auto:

class AutoModelForSpeechSeq2Seq(transformers.models.auto.auto_factory._BaseAutoModelClass)
 |  AutoModelForSpeechSeq2Seq(*args, **kwargs)
 |  
 |  This is a generic model class that will be instantiated as one of the model classes of the library (with a sequence-to-sequence speech-to-text modeling head) when created
 |  with the [`~AutoModelForSpeechSeq2Seq.from_pretrained`] class method or the [`~AutoModelForSpeechSeq2Seq.from_config`] class
 |  method.
 |  
 |  This class cannot be instantiated directly using `__init__()` (throws an error).
 |  
 |  Method resolution order:
 |      AutoModelForSpeechSeq2Seq
 |      transformers.models.auto.auto_factory._BaseAutoModelClass
 |      builtins.object
 |  
 |  Class methods defined here:
 |  
 |  from_config(**kwargs) from builtins.type
 |      Instantiates one of the model classes of the library (with a sequence-to-sequence speech-to-text modeling h

In [ ]:
pipe(path1)

{'text': '혹시 가지던 거 u?'}

In [ ]:
path1 = '//content//drive//MyDrive//salmonn//audio/output.wav'
result = processor(path1)
result = model(result)
segments_tiny = result["segments"]
segments_tiny

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


ValueError: ignored

In [ ]:
path1 = '/content/drive/MyDrive/salmonn/audio/ycyc.wav'
path2 = '/content/drive/MyDrive/salmonn/audio/test.mp3'
path3 = '/content/drive/MyDrive/salmonn/audio/fltest.flac'

pipe(path1) #wav
pipe(path2) #mp3
pipe(path3) #flac

{'text': '혹시 가지던 거 u?'}

In [ ]:
# print(pipe(path2))

{'text': '혹시 가지던 거 u?'}

In [ ]:
# print(pipe(path3))

{'text': '혹시 가지던 거 u?'}


In [ ]:
# import datasets
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm

path = '/content/drive/MyDrive/salmonn/audio/output.wav'

dataset = path

# KeyDataset (only *pt*) will simply return the item in the dict returned by the dataset item
# as we're not interested in the *target* part of the dataset. For sentence pair use KeyPairDataset
for out in tqdm(pipe(KeyDataset(dataset, "file"))):
    print(out)



  0%|          | 0/47 [00:00<?, ?it/s]

TypeError: ignored

In [ ]:
# 2. 한국어 학습 whisper '성범' Load model directly 파인튜닝 필요할 때
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq

processor = AutoProcessor.from_pretrained("SungBeom/whisper-base-ko")
model = AutoModelForSpeechSeq2Seq.from_pretrained("SungBeom/whisper-base-ko")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import torch
# from transformers import AutoProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

# model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en")

ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")


inputs = processor(ds[0]["audio"]["array"], return_tensors="pt")
input_features = inputs.input_features

generated_ids = model.generate(inputs=input_features)

transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)
transcription

Downloading:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

Dataset librispeech_asr downloaded and prepared to /root/.cache/huggingface/datasets/hf-internal-testing___librispeech_asr/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b. Subsequent calls will reuse this data.


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


['며칠 클 때가 지하철에 있는 미디엄 클래스예요? 그리고 저희가 여행 왔습니다. 민원 가수예요?']

In [ ]:
import soundfile as sf

path = '/content/ycyc.wav'  # 'path/to/your/file.mp3' 파일 경로로 바꿀 수 있습니다

# 오디오 파일 읽기 (파일 형식에 따라 다른 라이브러리 사용 가능)
audio_input, sample_rate = sf.read(path)

# 프로세서를 사용하여 모델이 이해할 수 있는 형식으로 오디오 데이터 변환
inputs = processor(audio_input, sampling_rate=16000, return_tensors="pt")
input_features = inputs.input_features

# 오디오 특징으로부터 토큰 ID 생성
generated_ids = model.generate(input_features)

# 생성된 토큰 ID를 텍스트로 디코드
transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)
print(transcription)

['아 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예 예']


In [ ]:
# Assuming 'generated_ids' is a list of lists, where each inner list contains token IDs for one transcription
transcriptions = processor.batch_decode(generated_ids, skip_special_tokens=True)

# Now 'transcriptions' is a list of strings, each string is a decoded transcription
for i, transcription in enumerate(transcriptions):
    print(f"Transcription {i+1}: {transcription}")


Transcription 1: 며칠 클 때가 지하철에 있는 미디엄 클래스예요? 그리고 저희가 여행 왔습니다. 민원 가수예요?


In [ ]:
#######################작동 안됨#######################
# import torch
# import torchaudio

# # 오디오 파일 경로 지정 (WAV 또는 MP3 파일 경로)
# path = '/content/ycyc.wav' # '.wav'를 '.mp3'로 변경 가능

# # 오디오 파일 로드
# speech_array, original_sampling_rate = torchaudio.load(path)

# target_sampling_rate = 16000
# if original_sampling_rate != target_sampling_rate:
#     resampler = torchaudio.transforms.Resample(orig_freq=original_sampling_rate, new_freq=target_sampling_rate)
#     speech_array = resampler(speech_array)

# # 오디오 데이터가 스테레오인 경우 모노로 변환
# if speech_array.shape[0] > 1:
#     speech_array = torch.mean(speech_array, dim=0, keepdim=True)

# # 리샘플링된 오디오 데이터가 모델이 요구하는 형태인지 확인
# # (1, time) 형태로 변환
# if speech_array.ndim == 2 and speech_array.shape[0] != 1:
#     speech_array = speech_array.unsqueeze(0)

# # 리샘플링된 오디오 파일 전처리
# inputs = processor(speech_array, sampling_rate=target_sampling_rate, return_tensors="pt", padding=True)

# # 모델을 사용하여 음성 인식 수행
# with torch.no_grad():
#     outputs = model.generate(inputs.input_values)

# # 결과 디코딩
# transcribed_text = processor.batch_decode(outputs, skip_special_tokens=True)

# # 결과 출력
# print(transcribed_text[0])
# # 여기에 타임스탬프를 얻는 추가 코드를 삽입합니다.
# # Whisper 모델은 디폴트로 타임스탬프를 제공하지 않습니다.
# # 만약 이러한 기능이 필요하다면, 추가적인 라이브러리나
# # 타임스탬프를 추출할 수 있는 다른 방법을 찾아야 합니다.


ValueError: ignored

In [ ]:
#Warning 뜨면 두번 실행
! pip install pyannote.audio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.8/553.8 kB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pyannote.audio
from pyannote.audio import Audio
from pyannote.core import Segment
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding

In [ ]:
pip install ffmpeg

  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6082 sha256=67b038de50b435525c789c49de83e920aea0863a914cc8b0719a2d578c41d174
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg


In [ ]:
pipe.dir()

AttributeError: ignored

In [ ]:
#필요하면 실행
import datetime #날짜와 시간을 조작하는 클래스를 제공합니다. 이를 통해 날짜와 시간에 대한 객체를 생성, 조작, 연산, 포매팅 및 파싱하는 기능
import torch
import subprocess #Python 스크립트에서 새로운 프로세스를 생성, 연결된 입/출력 파이프와 에러 파이프를 관리하고, 그 프로세스들의 리턴값을 얻을 수 있게 해주는 모듈

import wave #오디오 파일의 WAV 형식을 읽고 쓰기 위한 표준 라이브러리 이 모듈을 사용하여 WAV 파일의 헤더 정보를 읽거나 WAV 파일을 생성
import contextlib #컨텍스트 관리자를 만들고 사용하기 위한 유틸리티가 포함된 라이브러리입니다. 예를 들어, 파일이나 세션과 같이 'with' 블록으로 사용될 수 있는 리소스를 쉽게 관리

from sklearn.cluster import AgglomerativeClustering #데이터를 계층적으로 유사한 그룹으로 묶는 데 사용
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=device)

(…)a-voxceleb/resolve/main/hyperparams.yaml:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

(…)eleb/resolve/main/mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

(…)-voxceleb/resolve/main/label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

In [ ]:
#위스퍼
! pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-v1sq9rhf
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-v1sq9rhf
  Resolved https://github.com/openai/whisper.git to commit fcfeaf1b61994c071bba62da47d7846933576ac9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import whisper

In [ ]:
# model_large = whisper.load_model('large') #가장 큰 모델

100%|█████████████████████████████████████| 2.87G/2.87G [00:43<00:00, 70.2MiB/s]


In [ ]:
# model_base = whisper.load_model('base') #베이스 모델

100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 68.1MiB/s]


In [ ]:
model_tiny = whisper.load_model('tiny') #가장 작은 모델

In [ ]:
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
import torchaudio
import torch

# # 모델과 프로세서 로드
# processor = AutoProcessor.from_pretrained("SungBeom/whisper-base-ko")
# model = AutoModelForSpeechSeq2Seq.from_pretrained("SungBeom/whisper-base-ko")

# 오디오 파일 경로
audio_file_path = '/content/ycyc.wav'

# 오디오 파일 로드
speech_array, sampling_rate = torchaudio.load(audio_file_path)

# 모델에 맞는 샘플링 레이트로 리샘플링
if sampling_rate != processor.feature_extractor.sampling_rate:
    resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=processor.feature_extractor.sampling_rate)
    speech_array = resampler(speech_array)

# 샘플링 레이트를 모델의 기대값으로 설정
sampling_rate = processor.feature_extractor.sampling_rate

# 오디오 파일 전처리
inputs = processor(speech_array, sampling_rate=sampling_rate, return_tensors="pt", padding=True)

# 모델을 사용하여 음성 인식 수행
with torch.no_grad():
    outputs = model.generate(inputs.input_values)

# 결과 디코딩
transcribed_text = processor.batch_decode(outputs, skip_special_tokens=True)

# 결과 출력
print(transcribed_text[0])

ValueError: ignored

In [ ]:
result

{'text': ' 네 hi unio aub dedeov people 네, 오세요. 네, 오세요. 아, 그 오늘은. 아, 그 치소 외주셨다고 했고, 오세요. 네, 아, 마주스, 마주스, 마주스. 혹시 피연량이 어떻게도 드실까요? 네, 성윤관이에요, 그래요. 아, 네, 아기입니다. 네, 감사합니다. 네. 네, 감사합니다.',
 'segments': [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 29.16,
   'text': ' 네 hi unio aub dedeov people',
   'tokens': [50364,
    8808,
    4879,
    517,
    1004,
    257,
    836,
    368,
    1479,
    5179,
    561,
    51822],
   'temperature': 1.0,
   'avg_logprob': -5.761086684006911,
   'compression_ratio': 0.7837837837837838,
   'no_speech_prob': 0.056712955236434937},
  {'id': 1,
   'seek': 3000,
   'start': 30.0,
   'end': 40.0,
   'text': ' 네, 오세요.',
   'tokens': [50364, 8808, 11, 5175, 7046, 13, 50864],
   'temperature': 0.0,
   'avg_logprob': -0.9728947252330213,
   'compression_ratio': 1.5842696629213484,
   'no_speech_prob': 0.11809512972831726},
  {'id': 2,
   'seek': 3000,
   'start': 40.0,
   'end': 42.0,
   'text': ' 네, 오세요.',
   'tokens': [50864, 8808, 11, 5175, 7046, 13, 509

In [ ]:
segments_tiny

[{'id': 0,
  'seek': 0,
  'start': 0.0,
  'end': 29.16,
  'text': ' 네 hi unio aub dedeov people',
  'tokens': [50364,
   8808,
   4879,
   517,
   1004,
   257,
   836,
   368,
   1479,
   5179,
   561,
   51822],
  'temperature': 1.0,
  'avg_logprob': -5.761086684006911,
  'compression_ratio': 0.7837837837837838,
  'no_speech_prob': 0.056712955236434937},
 {'id': 1,
  'seek': 3000,
  'start': 30.0,
  'end': 40.0,
  'text': ' 네, 오세요.',
  'tokens': [50364, 8808, 11, 5175, 7046, 13, 50864],
  'temperature': 0.0,
  'avg_logprob': -0.9728947252330213,
  'compression_ratio': 1.5842696629213484,
  'no_speech_prob': 0.11809512972831726},
 {'id': 2,
  'seek': 3000,
  'start': 40.0,
  'end': 42.0,
  'text': ' 네, 오세요.',
  'tokens': [50864, 8808, 11, 5175, 7046, 13, 50964],
  'temperature': 0.0,
  'avg_logprob': -0.9728947252330213,
  'compression_ratio': 1.5842696629213484,
  'no_speech_prob': 0.11809512972831726},
 {'id': 3,
  'seek': 3000,
  'start': 42.0,
  'end': 44.0,
  'text': ' 아, 그 오늘은.'

In [ ]:
attributes = dir(model)
attributes

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_auto_class',
 '_backward_compatibility_gradient_checkpointing',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_check_and_enable_flash_attn_2',
 '_compiled_call_impl',
 '_convert_head_mask_to_5d',
 '_copy_lm_head_original_to_resized',
 '_create_repo',
 '_dispatch_accelerate_model',
 '_expand_inputs_for_generation',
 '_extend_attention_mask',
 '_extend_token_type_ids',
 '_extract_past_from_model_output',
 '_extract_token_timestamps',
 '_forward_hooks',
 '_forward_hooks_always_called',

In [ ]:
# path = '/content/ycyc.wav'
path = '/content/test.mp3'
result = model.generation_config(path)
result

TypeError: ignored

In [ ]:
generate

In [ ]:
path = '/content/ycyc.wav'
result = model_tiny.transcribe(path)
result
segments_tiny = result["segments"]
segments_tiny

[{'id': 0,
  'seek': 0,
  'start': 0.0,
  'end': 29.16,
  'text': ' 네 hi unio aub dedeov people',
  'tokens': [50364,
   8808,
   4879,
   517,
   1004,
   257,
   836,
   368,
   1479,
   5179,
   561,
   51822],
  'temperature': 1.0,
  'avg_logprob': -5.761086684006911,
  'compression_ratio': 0.7837837837837838,
  'no_speech_prob': 0.056712955236434937},
 {'id': 1,
  'seek': 3000,
  'start': 30.0,
  'end': 40.0,
  'text': ' 네, 오세요.',
  'tokens': [50364, 8808, 11, 5175, 7046, 13, 50864],
  'temperature': 0.0,
  'avg_logprob': -0.9728947252330213,
  'compression_ratio': 1.5842696629213484,
  'no_speech_prob': 0.11809512972831726},
 {'id': 2,
  'seek': 3000,
  'start': 40.0,
  'end': 42.0,
  'text': ' 네, 오세요.',
  'tokens': [50864, 8808, 11, 5175, 7046, 13, 50964],
  'temperature': 0.0,
  'avg_logprob': -0.9728947252330213,
  'compression_ratio': 1.5842696629213484,
  'no_speech_prob': 0.11809512972831726},
 {'id': 3,
  'seek': 3000,
  'start': 42.0,
  'end': 44.0,
  'text': ' 아, 그 오늘은.'

In [ ]:
from transformers import pipeline
from tqdm.auto import tqdm




# # tqdm을 사용하여 파이프라인 처리 과정을 반복하면서 결과를 출력합니다.
# for out in tqdm(pipe(audio_file_path)):
#     print(out)


# result = model('/content/test.mp3')
# result
# segments_tiny = result["segments"]
# segments_tiny
# result = pipe(audio_file_path)
# result
# for idx, audio_file in enumerate(audio_file_path):
#     # 오디오 파일을 파이프라인에 전달하고 결과를 받습니다.
#     result = pipe(audio_file_path)
#     # 결과와 함께 인덱스를 출력합니다.
#     print(idx, audio_file)

In [ ]:
# 오디오 파일 경로를 지정
path = '/content/ycyc.wav'

for idx, contents in enumerate(1, model_tiny(path)):
    # 오디오 파일을 파이프라인에 전달하고 결과를 받습니다.
    result = pipe(audio_file_path)
    # 결과와 함께 인덱스를 출력합니다.
    print(result, idx, contents)
    # print(idx, audio_file)

TypeError: ignored

In [ ]:
model_tiny.transcribe(audio_file_path)

RuntimeError: ignored

In [ ]:
print(model)

Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(80, 512, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0-5): 6 x ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=512, out_features=512, bias=True)
          (key): Linear(in_features=512, out_features=512, bias=False)
          (value): Linear(in_features=512, out_features=512, bias=True)
          (out): Linear(in_features=512, out_features=512, bias=True)
        )
        (attn_ln): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=512, out_features=2048, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=2048, out_features=512, bias=True)
        )
        (mlp_ln): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_post): LayerNorm((512,), eps=1e-05,

In [ ]:
result = pipe('/content/Audio_1.wav')

FileNotFoundError: ignored

- id: 이는 각 세그먼트에 대한 고유 식별자입니다. 이를 통해 세그먼트를 구별할 수 있습니다.
- seek: 오디오 파일 내에서 특정 세그먼트가 시작되는 바이트 오프셋을 나타냅니다. 간단히 말해서, seek 값은 오디오 스트림에서 해당 세그먼트가 시작하는 지점까지의 바이트 수를 의미합니다. 이는 오디오 파일 내에서 특정 위치를 찾거나 접근할 때 사용할 수 있습니다.seek 값이 5700인 경우, 오디오 파일의 5700번째 바이트부터 해당 세그먼트의 음성 데이터가 시작된다는 것을 의미합니다. 이 정보는 오디오 파일을 재생하거나 특정 부분을 추출할 때 유용하게 사용될 수 있습니다.
- start: 세그먼트가 시작되는 시간(초)을 나타냅니다.
- end: 세그먼트가 끝나는 시간(초)을 나타냅니다.
- text: 음성 인식을 통해 변환된 텍스트입니다.
- tokens: 텍스트를 나타내는 토큰의 시퀀스입니다. 토크나이징 과정에서 원본 텍스트가 모델이 이해할 수 있는 토큰으로 변환됩니다.
- temperature: 이는 생성 과정에서 사용된 온도 매개변수를 나타냅니다. 온도는 모델이 얼마나 창의적으로 텍스트를 생성할지를 조절하는 데 사용되며, 여기서는 0으로 설정되어 있습니다.
- avg_logprob: 평균 로그 확률은 모델이 각 토큰을 얼마나 확신하는지를 나타내는 지표입니다. 일반적으로 더 높은 값은 더 높은 확신을 의미합니다.
- compression_ratio: 이 값은 오디오의 압축 비율을 나타냅니다. 음성 인식 시스템에서 원본 오디오 데이터와 추출된 텍스트 데이터 사이의 정보량 비율을 나타낼 수 있습니다.
- no_speech_prob: 이는 해당 세그먼트에 음성이 없을 확률을 나타냅니다. 값이 높을수록 해당 세그먼트에서 음성이 없을 가능성이 높다는 것을 의미합니다.

In [ ]:
# 텍스트만 출력 'text' key값의 value인 텍스트 출력, "segments"는 각 문장의 정보와 함께 출력
result

{'text': ' 또 뭐지? 같이 잊을 때 쓴 거 아니야 죄송해? 아니야 왜 이래? 처음 십쪄까지는 조용히 있어 왜 이래? 제일 많이 들을 거기 때문에 진짜 본인 목소리를 들으면 인정 이렇게 아까가 형이 전화했던 것 중에 막판에 목소리가 겹쳐나 봐요 재밌다 여자 형 나오는 거 확인했습니다 이거 여기 시간대실 예를 들어요 저거는 거구나 이게 시간이거든요 만약에 자막에 회고 표시할 거는 시간보고 표시하면 돼요 그러니까 도언님이 그렇죠 비비티에 만들어놨어요 자막 가는 거 아빌? 네 네 제가 상한 그런 거 자막가 부를 뻔해라 그랬으니까 그거는 두 개가 두 개가 김지에 말하는 건 아니고 따로 말하는데 자막에 두 개가 붙어있거든요 그래서 엔버가 말할 때는 엔버가 따로 뜨고 왜 이래가 말할 때 왜 이래 동시에 말해도 동시에 말하더라도 진짜 사운드 물릴 때 토론을 두 개가 이렇게 해야겠네 네 그리고 이게 솔직히 저희가 그냥 귀 가리고 영화 봤을 때는 웬만한 건 다 이해 되거든요 근데 그래도 이제 이해 안 되는 부분이 가끔 있을 거고 소리가 안 들리는다는 가정 하면 있는 게 훨씬 보기 편할 거에요 달려 있는 게 영화가 배경은 설명이나 나레이션이 말한다 어떤 사람이 말하는 건 자막에다 표시되거든요 제가 그렇게 정한 게 아니라 배 리허프리 영화에 다 그렇게 나와요 그래서 나는 이유도 있다 살통력도 있다 근데 자막에 생성할 때 사람들이 전사 작업하는데 그때 사람이 보고 판단해서 지금 쾌움이 나 배경을 막히나 상황에 대해서 전사를 한다고 사람을 직접 판단해서 근데 배경을 막히나 배경을 막히나 배경을 막히나 배경을 막히나 배경을 막히나 배경을 막히나 두개 합쳐서 할 수 있으면 거과자면 좋다 두개 어떤 거 하고 싶는지 손걸치도 괜찮긴 한데 이거 완전 제 생각에는 비중이 애께로 하는 것 같아요 그거야 할 거랑 처음에서 막 다 배워야 될 것 같긴 해요 그런다면 두명은 집중한 느낌의 조글랑 여성의 그런 쪽에 신경쓰고 나눠서 했다가 무조건 거를 합쳐서 진행하는 쪽으로 해야 될 것 같아요 그걸

In [ ]:
result = model.transcribe('/content/test.mp3')
result
segments_tiny = result["segments"]
segments_tiny
# segments_large = result["segments"]

r

[{'id': 0,
  'seek': 0,
  'start': 0.0,
  'end': 2.0,
  'text': ' 또 뭐지?',
  'tokens': [50364, 7992, 7034, 1831, 30, 50464],
  'temperature': 0.0,
  'avg_logprob': -0.7496095445421007,
  'compression_ratio': 1.3316582914572865,
  'no_speech_prob': 0.14222994446754456},
 {'id': 1,
  'seek': 0,
  'start': 2.0,
  'end': 4.0,
  'text': ' 같이 잊을 때 쓴 거 아니야',
  'tokens': [50464,
   16358,
   1332,
   232,
   1638,
   7765,
   11647,
   112,
   3675,
   20425,
   50564],
  'temperature': 0.0,
  'avg_logprob': -0.7496095445421007,
  'compression_ratio': 1.3316582914572865,
  'no_speech_prob': 0.14222994446754456},
 {'id': 2,
  'seek': 0,
  'start': 4.0,
  'end': 6.0,
  'text': ' 죄송해?',
  'tokens': [50564, 41939, 5302, 30, 50664],
  'temperature': 0.0,
  'avg_logprob': -0.7496095445421007,
  'compression_ratio': 1.3316582914572865,
  'no_speech_prob': 0.14222994446754456},
 {'id': 3,
  'seek': 0,
  'start': 6.0,
  'end': 8.0,
  'text': ' 아니야',
  'tokens': [50664, 20425, 50764],
  'temperature': 0.

In [ ]:
whisper --help

NameError: ignored

In [ ]:
segments_large

[{'id': 0,
  'seek': 0,
  'start': 0.0,
  'end': 5.5,
  'text': ' 다시 됐습니다. 죄송해요.',
  'tokens': [50364, 15463, 16718, 3115, 13, 41939, 15788, 13, 50639],
  'temperature': 0.0,
  'avg_logprob': -0.5603539784749348,
  'compression_ratio': 1.2542372881355932,
  'no_speech_prob': 0.06571722030639648},
 {'id': 1,
  'seek': 0,
  'start': 5.5,
  'end': 10.0,
  'text': ' 아니요. 왜 왜 왜 이러고 있어요.',
  'tokens': [50639,
   5651,
   1495,
   13,
   9883,
   9883,
   9883,
   37398,
   1313,
   12654,
   13,
   50864],
  'temperature': 0.0,
  'avg_logprob': -0.5603539784749348,
  'compression_ratio': 1.2542372881355932,
  'no_speech_prob': 0.06571722030639648},
 {'id': 2,
  'seek': 0,
  'start': 10.0,
  'end': 14.0,
  'text': ' 처음 10초까지는 조용히 있어야 돼요.',
  'tokens': [50864,
   18736,
   1266,
   27005,
   8786,
   1098,
   7430,
   8761,
   5905,
   2297,
   21637,
   21565,
   13,
   51064],
  'temperature': 0.0,
  'avg_logprob': -0.5603539784749348,
  'compression_ratio': 1.2542372881355932,
  'no_speech_

In [ ]:
segments

[{'id': 0,
  'seek': 0,
  'start': 0.0,
  'end': 3.54,
  'text': ' 또 같이 됐습니다.',
  'tokens': [50364, 7992, 16358, 16718, 3115, 13, 50541],
  'temperature': 0.0,
  'avg_logprob': -0.8062918736384466,
  'compression_ratio': 1.2657004830917875,
  'no_speech_prob': 0.14291627705097198},
 {'id': 1,
  'seek': 0,
  'start': 3.54,
  'end': 5.5,
  'text': ' 죄송해요?',
  'tokens': [50541, 41939, 15788, 30, 50639],
  'temperature': 0.0,
  'avg_logprob': -0.8062918736384466,
  'compression_ratio': 1.2657004830917875,
  'no_speech_prob': 0.14291627705097198},
 {'id': 2,
  'seek': 0,
  'start': 5.5,
  'end': 7.12,
  'text': ' 아니 아니.',
  'tokens': [50639, 5651, 5651, 13, 50720],
  'temperature': 0.0,
  'avg_logprob': -0.8062918736384466,
  'compression_ratio': 1.2657004830917875,
  'no_speech_prob': 0.14291627705097198},
 {'id': 3,
  'seek': 0,
  'start': 7.12,
  'end': 10.32,
  'text': ' 왜 왜 이러고 있어요?',
  'tokens': [50720, 9883, 9883, 37398, 1313, 12654, 30, 50880],
  'temperature': 0.0,
  'avg_logprob':

In [ ]:
# ASR 파이프라인을 초기화합니다.
# pipe = pipeline("automatic-speech-recognition", model="SungBeom/whisper-base-ko")

# 오디오 파일을 로드합니다. 예를 들어 'sample.wav'를 사용하세요.
# Colab에서 파일을 업로드하려면 다음 코드를 사용할 수 있습니다:
# from google.colab import files
# uploaded = files.upload()

path = '/content/Audio_1.wav' # 실제 오디오 파일 경로로 변경하세요.

# 오디오 데이터를 로드합니다. 오디오 파일은 단일 채널 (모노)이고 적절한 샘플링 레이트를 가져야 합니다.
waveform, sample_rate = torchaudio.load(path)

# 오디오 데이터를 파이프라인에 전달하여 텍스트로 변환합니다.
transcription = pipe(waveform.squeeze(), sampling_rate=sample_rate)

# 변환된 텍스트를 출력합니다.
print(transcription['text'])

In [ ]:
with contextlib.closing(wave.open('/content/Audio_1.wav','r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)

In [ ]:
audio = Audio()

In [ ]:
# if path[-3:] != 'wav':
#   subprocess.call(['ffmpeg', '-i', r'/content/Audio_1.wav', 'audio.wav', '-y'])
#   path = '/content/'

IndentationError: ignored

In [ ]:
path = '/content/Audio_1.wav'

In [ ]:
def segment_embedding(segment):
    start = segment["start"]
    # Whisper overshoots the end timestamp in the last segment
    end = min(duration, segment["end"])
    clip = Segment(start, end)
    waveform, sample_rate = audio.crop(path, clip)

    # Reshape the waveform tensor to match the expected input shape
    waveform = waveform.unsqueeze(0)

    return embedding_model(waveform)

In [ ]:
embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
    embeddings[i] = segment_embedding(segment)


embeddings = np.nan_to_num(embeddings)

AssertionError: ignored

In [ ]:
clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments)):
  segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

In [ ]:
num_speakers = 5

language = 'Korean'

model_size = 'base'


model_name = model_size
if language == 'Korean' and model_size != 'base':
  model_name += '.ko'
